Import Important Libraries and Read in the graph

In [ ]:
import networkx as nx
import numpy as np
import matplotlib
import matplotlib.pylab as plt
%matplotlib inline
import random as rnd
rnd.seed()
G = nx.read_graphml('graph.graphml')

You can try to plot the graph if you want but since this is probably a massive graph, It might take up to much memory

In [ ]:
nx.draw(G)
plt.show()

Here we are getting the the number of nodes and edges and the mean degree. You can use the number of nodes and edges to check that you graphml file was exported and read correctly. we also get the mean degree which will provide a littel bit of info about the degree distribution of the nodes in the network.

In [ ]:
G.nodes
n = G.number_of_nodes()
m = G.number_of_edges()
kmean = (2*m)/n

print(f'number of nodes, n  = {n}')
print(f'number of edges, m  = {m}')
print(f'mean degree,    <k> = %5.2f' % kmean)

Here we can vizualize what the degree distribution is looking like in the network for both the in and out degrees. This helpful in getting an idea of the connectivness of the network and deteting if there are any nodes that are more highly conneted than others. These highly connected nodes are the nodes we are interested in.

In [ ]:
def plot_2CCDF(kins,kouts):
    # input : two lists of in- and out-degrees
    # output: a plot showing CCDFs of the in- and out-degree distributions Pr(K>=k) for k>=1
    
    kin_max  = max(kins)
    kout_max = max(kouts)

    # histograms
    icounts, ibins = np.histogram(kins, bins=[i for i in range(kin_max+2)], density=True)
    icumcounts = np.cumsum(icounts)
    icumcounts = np.insert(icumcounts,0,0)
    ocounts, obins = np.histogram(kouts, bins=[i for i in range(kout_max+2)], density=True)
    ocumcounts = np.cumsum(ocounts)
    ocumcounts = np.insert(ocumcounts,0,0)

    # plots
    fig = plt.figure()
    ax1 = fig.add_subplot(111) # put multiple 
    plt.loglog(obins[1:-1], 1-ocumcounts[1:-1], 'bo', alpha=0.5, label='out-degree')
    plt.loglog(ibins[1:-1], 1-icumcounts[1:-1], 'rs', alpha=0.5, label='in-degree')
    plt.title('CCDF, in- and out-degrees (loglog)')
    plt.xlabel('Degree, k')
    plt.ylabel('Pr(K>=k)')
    plt.legend(loc='upper right');
    plt.show()
    return

In [ ]:
n = G.number_of_nodes()
m = G.number_of_edges()
# print(G.out_degree())
kin_mean = (m)/n
kout_mean = (m)/n
dgh = nx.degree_histogram(G)
md = 0
sum = 0
ind = 0


lto = G.out_degree()
l = len(lto)
nlto = []
for i in range(l):
    
    nlto.append(lto[i])
    
ilto = G.out_degree()
l = len(lto)
inlto = []
for i in range(l):
    inlto.append(ilto[i])
    
lti = G.in_degree()
l = len(lti)
nlti = []
for i in range(l):
    nlti.append(lti[i])
md = 0
sum = 0
ind = 0
hsum = 0
for i in range(len(inlto)):
    hsum = hsum + inlto[i]
while(sum < (hsum/2)):
    ind = ind +1
    l = len(dgh)
    m = max(dgh)
    sum = sum + m
    j = 0
    for i in range(l-1):
        if m == inlto[i]:
            inlto.pop(i)

            
print(f'number of nodes, n  = {n}')
print(f'number of edges, m  = {m}')
print(f'\nmean(k_in)  = %5.2f' % kin_mean)
print(f'mean(k_out) = %5.2f' % kout_mean)
print(f'\nsmallest num for 50%  = {ind} of {n} nodes')
plot_2CCDF(nlti,nlto)

These statistics will again provide us information on how connected this network is and how well/quickf information would move through the network.

In [ ]:
diameter = nx.diameter(G)
ellmean = nx.average_shortest_path_length(G)
C = nx.transitivity(G)
h = nx.number_connected_components(G)
print(f'diameter = {diameter}')
print(f'mean geodesic distance, <ell> = %5.2f' % ellmean)
print(f'clustering coefficient, C     = %5.2f' % C)
print(f'number of components,   h     =  {h}')

Here we check the obustness of the network with three applicable metrics from the paper: "The Rationality of Four Metrics of Network Robustness: A Viewpoint of Robust Growth of Generalized Meshes" found here https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0161077

Below are the docs for the three metrics for more information:

Algebraic Connectivity: https://networkx.org/documentation/stable/reference/generated/networkx.linalg.algebraicconnectivity.algebraic_connectivity.html

Betweeness Centrality: https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.edge_betweenness_centrality.html

Global Efficency: https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.efficiency_measures.global_efficiency.html#networkx.algorithms.efficiency_measures.global_efficiency

In [ ]:
ac = nx.algebraic_connectivity(G)
ec = nx.edge_betweenness_centrality(G)
ge = round(nx.global_efficiency(G), 5)

print(f'algebraic connectivity = {ac}')
print(f'edge betweeness centrality = {ec}')
print(f'Global efficency = {ge}')

The following networkx functions are all viable measure to look at the centrality of nodes. These will be usefull in identifying the big players in the network so that we can futher study what makes them so essential to the network. the voterank function at the bottom provides a list of the top most influential nodes in the network which again provides us information similar to the other statistics.

In [ ]:
idc = nx.in_degree_centrality(G)
odc = nx.out_degree_centrality(G)
cfcc = nx.current_flow_closeness_centrality(G)
bc = nx.betweenness_centrality(G)
lc = nx.load_centrality(G)
vr = nx.voterank(G)

Below are a couple of helper functions used to check for and find nodes in the graph. Below that is a script you can run to query a subgraph from the graphml file so that you can run this network analysis on a smaller frame of reference if desired.

In [ ]:
def findNodeInGraph(G, ID):
    for node in G.nodes:
        if node == ID:
            return node
           
def isNodeInGraph(G, ID):
    for node in G.nodes:
        if node == ID:
            return True
        else:
            return False

In [ ]:
Id = ""
r = 1


n = findNodeInGraph(G, Id)
subG = nx.ego_graph(G, n, r, True, True, None)

nx.draw(subG)
plt.show()

Here we have the Degree Corrected Stochasitc Block Model which is detailed in this paper :
http://www.stat.yale.edu/~hz68/DCBM-aos.pdf

This Algorithm will take a long time to run and will require a lot of computational resources, but it is a community detection algorithm that uses the optimization of log likelyhood function to find the correct partitions and correctly identify communities. This code is from some of my coursework so this should be used as a tool and less to be built into a the product itself.

In [ ]:
def random_z(n,c):
    # input  : number of nodes n, and number of groups c
    # output : returns a random partition in a dictionary z, where z_i = Uniform(0,c-1)

    import random as rnd
    rnd.seed()
    
    z = dict()

 

    for i in range(n):
        z.update({i: np.random.choice(c, 1)[0]})
    


    return z

def tabulate_wk(G,z,c):
    # This function tabulates the w_rs and kappa_r auxiliary data structures for the DC-SBM
    #
    # input  : G is simple graph with n nodes
    #        : z is a dictionary of group labels for G's nodes, into c groups
    #        : c is scalar, number of possible groups
    # output : wrs, kpr
    # 
    # WARNING: function is optimistic: assumes inputs are properly formatted
    
    wrs = np.zeros([c,c]) # count of stubs from group r to group s
    kpr = np.zeros([c,1]) # total degree of group r

    
    
#     calculated wrs by looping through edges to see if dictionary between the two is same or differnt. 
#     If same it add 2 to that index to account for stubs on both sides of edge. if different adds one to each of the indexs where the groups appear.
    for e in list(G.edges()):
        if z[e[0]] == z[e[1]]:
            wrs[z[e[0]]][z[e[1]]]+= 2
        else:
            wrs[z[e[0]]][z[e[1]]]+= 1
            wrs[z[e[1]]][z[e[0]]]+= 1
#     Calculates kpr by summing each column in the wrs matrix and updates the index at that column index
    for i in range(c):
        summ = 0
        for j in range(c):
            summ = summ + wrs[i][j]
        kpr[i]=summ



    return wrs,kpr

def dcsbm_LogL(wrs,kpr):
    # This function calculates the log-likelihood of the degree-corrected stochastic block model (DC-SBM)

    #
    # input  : wrs is a c x c np.array of stub counts
    #        : kpr is a c x 1 np.array of stub counts 
    # output : the dcsbm log-likelihood
    # 
    # WARNING: function is optimistic: assumes inputs are properly formatted

    c = wrs.shape[1]  # number of groups
    
    logL = 0
    for r in range(c):
        for s in range(c):
            if wrs[r,s] < 1 or kpr[r] < 1 or kpr[s] < 1:
                temp = 0 # define 0^0 = 1
            else:
                temp = wrs[r,s]*np.log( wrs[r,s] / (kpr[r]*kpr[s]) )
            logL = logL + temp
    
    return logL

def makeAMove(G,z,c,f):
    # For each non 'frozen' node in the current partition, this function tries all (c-1) possible group moves for it
    # It returns the combination of [node i and new group r] that produces the best log-likelihood over the non-frozen set
    # input  : G, a graph
    #        : z, a partition of G's nodes
    #        : c, the number of groups
    #        : f, a binary labeling of frozen nodes
    # output : bestL, the best log-likelihood found
    #        : bestMove, [i,r] the node i and new group r to achieve bestL
    
    bestL = -np.inf            # the best log-likelihood over all considered moves
    for i in G.nodes():        # loop over all nodes i
        if f[i] == 0:          # if i is not frozen
            s = int(z[i])      #  get current label of i
            for r in range(c): #  then loop over all groups r
                # print(f'v[{i}] s = {s}, r={r}, {r!=s}') # for debugging
                
                 
#               Checks to make sure old group is not same as new group then copies z and updates z so node is in new group.
                if r!=s:
                    zp = z.copy()
                    zp.update({i: r})
#               Runs Tabulate function and calculates Log likelyhood value and if its better than the previous best it saves the new one as the new best.
                    wrs,kpr = tabulate_wk(G,zp,c)
                    ll = dcsbm_LogL(wrs,kpr)
                    if ll > bestL:
                        bestL = ll
                        bests = s
                        bestMove = [i,r]
    # print(f'v[{bestMove[0]}] g[{int(bests)}] --> g[{bestMove[1]}] : {bestL}')
                
                
                
    return bestL,bestMove

def run_OnePhase(G,z0,c):
    # Runs one phase, initialized from partition z0
    # Returns the best partition found in the phase and the list of LogL values for all the phase's partitions
    # input  : G, a graph
    #        : z0, initial partition of G's nodes
    #        : c, the number of groups
    # output : zstar, the best partition of the phase
    #        : Lstar, the LogL of zstar
    #        : LL, the inorder list of LogL values for the n+1 partitions of this phase
    #        : halt, 1 if zstar=z0 (no better partition found)

    import copy      # for copy.deepcopy() function
    n    = G.order() # n, number of nodes
    LL   = []        # stores log-likelihoods over the entire algorithm (via .append)
    halt = 0         # flag: =0 if Lstar > L0 at the end of the phase; =1 if Lstar <= L0

    # initialize the phase
    wrs,kpr = tabulate_wk(G,z0,c)      # wrs, kpr, initial DC-SBM parameters
    L0      = dcsbm_LogL(wrs,kpr)      # store initial DC-SBM log-likelihood
    LL.append(L0)                      # track log-likelihood

    f     = dict.fromkeys(range(n), 0) # initially, all nodes unfrozen (tricky python)
    t     = 0                          # number of frozen nodes in this phase
    Lstar = L0                         # initially, z0 has the best LogL
    zstar = copy.deepcopy(z0)          # and z0 is the best partition
    tstar = t                          # tstar = 0

    # loop over all the nodes in G, making greedy move for each
    zt = copy.deepcopy(z0)             # start the loop at z0
    for j in range(n):
        # print(f'step {j}') # for debugging
        # print(f)

#       Gets best move and updates f to freeze the node moved and then copies zt and updates the copy of zt with the best move
        choiceL,choiceMove = makeAMove(G,zt,c,f)
        f.update({choiceMove[0]: 1})
        zt = copy.deepcopy(zt)
        zt.update({choiceMove[0]: choiceMove[1]})
#       Adds log likelyhood value to LL list and then if the new LL is better than previous best LL then update the and save these best values as Lstar and Zstar
        LL.append(choiceL)
        if choiceL > Lstar:
            Lstar = choiceL
            zstar = copy.deepcopy(zt)
        if Lstar == L0:
            halt = 1
        ##### do not modify below here #####

    return zstar,Lstar,LL,halt

    
    
    
def fit_DCSBM(G,c,T):
    # Runs the full locally greedy heuristic, with c groups
    # Returns the best partition found, its LogL, and the list of LogL values for all partitions considered
    # input  : G, a graph
    #        : c, the number of groups
    #        : T, the number maximum number of phases allowed
    # output : zstar, the best partition of the phase
    #        : Lstar, the LogL of zstar
    #        : LL, the inorder list of all LogL values considered
    #        : pc, the number of phases in LL

    import copy # for copy.deepcopy()
    
    # 1.0 locally greedy heuristic setup
    n  = G.order() # n, number of nodes
    LL   = []      # log-likelihoods over the entire algorithm (concat via .extend)
    halt = 0       # convergence flag

    # 2.0 generate initial partition, calculate wrs,kpr, and store the loglikelihood in Lt
    zt      = random_z(n,c)       # z0, initial partition
    wrs,kpr = tabulate_wk(G,zt,c) # wrs, kpr, initial DC-SBM parameters
    Lt      = dcsbm_LogL(wrs,kpr) # store initial DC-SBM log-likelihood

    # 3.0 the main loop
    pc = 0  # counter for number of phases completed
    while not halt:
        # 3.1 visualization of this phase's initial partition
        # print(f'phase[{pc}] z[0], logL = {Lt}')
        # drawGz(G,zt)



        zstar,Lstar,PLL,halt = run_OnePhase(G,zt,c)
        zt = copy.deepcopy(zstar)
        Lt = Lstar
        LL.extend(PLL)
        pc = pc + 1
        if pc == T:
            return
       
    
    print(f' --> WE HAVE CONVERGENCE <-- ') # a friendly alert
    return zstar,Lstar,LL,pc

def drawGz(G,z):
    
    # This function draws G with node labels from partition z
    #
    # input  : G is a networkx graph
    #        : z is a dictionary of group labels for G's nodes
    # output : none
    # 
    # WARNING: function is optimistic: assumes inputs are properly formatted

    colors = ['#d61111','#11c6d6','#d67711','#11d646','#1b11d6','#d611cc'] # map node labels to colors (for the visualization)

    node_colors = []
    for i in G.nodes():
        node_colors.append(colors[int(z[i])])
    nsize  = 600
    flabel = True

    if G.order() > 50:
        nsize  = 100
        flabel = False
        
    nx.draw_networkx(G,with_labels=flabel,node_size=nsize,width=2,node_color=node_colors) # draw it pretty
    limits=plt.axis('off')                                      # turn off axes
    plt.show() 

    return

In [ ]:
Go = nx.convert_node_labels_to_integers(G) # map node names to integers (0:n-1) [because indexing]

c    = 2       # c, number of groups
T    = 30      # maximum number of phases; HALT if pc >= T
reps = 20     # number of repetitions of fit_DCSBM()

Lbest = -np.inf
Zbest = ([])


# Loops over number of reps and finds best LogLikelyhood value returned by fit_DCSBM and draws the resulting network.
for i in range(reps):
    zstar,Lstar,LL,pc = fit_DCSBM(Go,c,T)
    if Lstar > Lbest:
        Lbest = Lstar
        Zbest = zstar
drawGz(Go,Zbest)
print("Best LL:", Lbest)

Here we have the Local Smoothing algorithm detailed by this paper: https://proceedings.neurips.cc/paper/2021/file/a9eb812238f753132652ae09963a05e9-Paper.pdf

This algorithm provides node attribute prediction based on the nodes it is in connection with. This code is from some of my coursework so this should be used as a tool and less to be built into a the product itself.

In [ ]:
def predictLabel_baseline(x):
    # input:  x, dict of observed labels
    # output: baseline predictor, Uniform(\vec{x}-\emptyset)
    non_missing = []
    for i in x:
        if x[i]!= -1:
            non_missing.append(x[i])
    return np.random.choice(non_missing)

def predictLabel_local(G,i,x,flag):
    
    # input:  G, simple networkx graph
    #         i, a node in G whose label we will predict
    #         x, dict of observed labels for G
    #         flag, binary value
    # output: local smoothing predictor output for i
    #         a print statement (see instructions) if flag=1
    if x[i] == -1:
        counts = {}
        neighbors = [n for n in G.neighbors(i)]
        neighbor_attributes = []
        for j in neighbors:
            if x[j] != -1:
                if not x[j] in neighbor_attributes:
                    neighbor_attributes.append(x[j])
                    counts[x[j]] = 1
                else: 
                    counts[x[j]]+=1
        if neighbor_attributes:
            r = [g for g,l in counts.items() if l==max(counts.values())]
            if len(r) == 1:
                if flag == 1:
                    print("node",i, ": -1 -> ", r[0], "(Smoothing)")
                return r[0]
            else:
                r = np.random.choice(r)
                if flag == 1:
                    print("node",i, ": -1 -> ", r[0], "(Smoothing)")
                return r
        else:
            base = predictLabel_baseline(x)
            if flag == 1:
                print("node",i, ": -1 -> ", base, "(Baseline)")
            return(base)
    else:
        return

In [ ]:
xp = x.copy()
for i in x:
    new_Att = predictLabel_local(Go,i,x,1)
    if new_Att != None:
        xp[i] = new_Att